In [2]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

In [3]:
import dagshub

dagshub.init(repo_owner='sourav664', repo_name='mlops-mini-project', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/sourav664/mlops-mini-project.mlflow')

Accessing as sourav664

Initialized MLflow to track repo "sourav664/mlops-mini-project"

Repository sourav664/mlops-mini-project initialized!

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
C:\Users\soura\AppData\Local\Temp\ipykernel_16576\4275467378.py:30: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [6]:
df = normalize_text(df)

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


C:\Users\soura\AppData\Local\Temp\ipykernel_16576\468518138.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [8]:
mlflow.set_experiment("Bow vs TfIdf")

2025/01/06 19:22:30 INFO mlflow.tracking.fluent: Experiment with name 'Bow vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/947550e06a6548b6a36519689665026e', creation_time=1736171549642, experiment_id='1', last_update_time=1736171549642, lifecycle_stage='active', name='Bow vs TfIdf', tags={}>

In [9]:
# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

In [10]:
# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [12]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # # Save and log the notebook
                # mlflow.log_artifact(__file__)
                import os
                notebook_path = "exp2_bow_vs_tfdf.ipynb"
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2025/01/06 19:24:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7951807228915663
Precision: 0.7858527131782945
Recall: 0.7990147783251231
F1 Score: 0.7923790913531998
🏃 View run LogisticRegression with BoW at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1/runs/569ded8af0044eeab96818367f8604d7
🧪 View experiment at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1


2025/01/06 19:24:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7946987951807228
Precision: 0.7791469194312797
Recall: 0.8098522167487685
F1 Score: 0.7942028985507247
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1/runs/922ef65e4d604d4ea79e54ca3f9f6769
🧪 View experiment at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1


2025/01/06 19:24:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Algorithm: RandomForest, Feature Engineering: TF-IDF
Accuracy: 0.7653012048192771
Precision: 0.7682926829268293
Recall: 0.7448275862068966
F1 Score: 0.7563781890945472
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1/runs/095be5b94a1845ce92568a623a31aa31
🧪 View experiment at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1


2025/01/06 19:28:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.7281927710843373
Precision: 0.8043184885290149
Recall: 0.587192118226601
F1 Score: 0.6788154897494305
🏃 View run GradientBoosting with BoW at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1/runs/d7c60a0e9c2d4411b3dff143406c42bb
🧪 View experiment at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1


2025/01/06 19:29:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.7253012048192771
Precision: 0.8068965517241379
Recall: 0.5763546798029556
F1 Score: 0.6724137931034483
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1/runs/4c28f8c8312d442faaa01d72f0147bd4
🧪 View experiment at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1
🏃 View run All Experiments at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1/runs/31d7166d16e84ae0ab3c020661377a70
🧪 View experiment at: https://dagshub.com/sourav664/mlops-mini-project.mlflow/#/experiments/1
